<h1>PL/SQL</h1>

## Contents:
* [Anonymous Blocks](#section1)
    * [Comments](#section1.1)
    * [Nesting Blocks](#section1.2)
    * [Scope of Variables](#section1.3)
    * [Visibility of Variables](#section1.4)
* [Data Types](#section2)
    * [Numbers](#section2.1)
    * [Inherited Types](#section2.2)
    * [Timestamps](#section2.3)
    * [Intervals](#section2.4)
    * [Booleans](#section2.5)
    * [Records](#section2.6)
    * [%ROWTYPE](#section2.7)
* [Loops](#section3)
    * [For Loop](#section3.1)
    * [Loop Labels](#section3.2)
    * [Nested Loops](#section3.3)
        * [Continue When](#section3.3.1)
    * [While Loops](#section3.4)
        * [Random Values](#section3.4.1)
* [Conditional Execution](#section4)
    * [If, Elsif, and Else Statements](#section4.1)
    * [When Statements](#section4.2)
    * [Simple Case Expression](#section4.3)
* [Cursors](#section5)
    * [Implicit Cursors](#section5.1)
    * [Explicit Cursors](#section5.2)


<h2>Anonymous blocks</h2> <a class="anchor" id="section1"></a>

Anonymous blocks are made up of three distinct sections. 
1. Declaration (optional)
2. Exectution (required)
3. Exception (optional)

In [2]:
-- declaration
declare
    l_counter number;
-- execution
begin
    l_counter := 1;
-- exception 
exception
    when others then
        null;
    end;

Error: Please load a database to perform operations

<h3>Comments</h3> <a class="anchor" id="section1.1"></a>

1. Single line: -- text here
2. Multiline: /* text 

                   also
                   
                    here */

<h3>Nesting Blocks</h3> <a class="anchor" id="section1.2"></a>

Blocks have the ability to be nested.

In [4]:
declare
    l_outer number;
begin
    l_outer := 1; --initialization of l_var
    
    declare
        l_inner number;
    begin
        l_inner := 2/0;
    exception
        when others then
            --do something
        null;
    end;
    
    l_outer := 2;
    
exception
    when others then
        -- do something
    null;
end;

Error: Please load a database to perform operations

<h3>Scope of Variables</h3> <a class="anchor" id="section1.3"></a>

Variables defined in an outer block, have a scope in the outer block and all inner blocks it contains (similar to Global variables in Python). 
Variables defined in an inner block only have a scope inside of the inner block (similar to a locally defined variable in a function).

In [5]:
declare
    l_outer number;
begin
    l_outer := 1; --initialization of l_var
    
    declare
        l_inner number;
    begin
        l_outer := 2; --update l_outer value
        l_inner := 3; --inside scope of inner block
    end;

exception
    when others then
        --do something
    null;
end;

Error: Please load a database to perform operations

<h3>Visibility of Variables</h3> <a class="anchor" id="section1.4"></a>

If a variable is defined in bouth an outer and inner block, the inner block will override the visibility of the outer block variable.

In [7]:
declare
    l_var number;
    l_outer number;
begin
    l_outer := 1;
    l_var := 1;
    
    declare
        l_var number;
        l_inner number;
    begin
        l_var := 2;
        l_inner := 2;
        dbms_output.put_line('l_var in the innner block is '||l_var);
    end;
    
    dbms_output.put_line('l_var in the outer block is '||l_var);
exception
    when other then
        --do something
        null;
end;

Error: Please load a database to perform operations

<h2>Data Types</h2> <a class="anchor" id="section2"></a>

<h3>Numbers</h3> <a class="anchor" id="section2.1"></a>

In [8]:
/*
Declaring different number types.
number(5,2) --> number(scale, precision) where:
	scale = number of digits in number, precision = accuracy passed decimal (if applicable)
If number of digits is greater than precision + scale, error is returned.
	Ex: 1234.56 number(5, 2) returns "PL/SQL: numeric or value%s error"
	Ex: 1234.56 number(5, -2) returns 1200 (rounds 2 places to the left of decimal)
*/
declare
    l_num number;
    l_num_constant constant number := 5;
    l_num_default number(5, 2) default
    l_num_float number;
begin
    l_num_float := 3.245;
    dbms_output.put_lin('l_num: '||l_num);
end;

Error: Please load a database to perform operations

<h3>Inherited Types</h3> <a class="anchor" id="section2.2"></a>

In [9]:
/*
Inherited types
Below, l_num_vartype and l_num_coltype are inheriting the number type of l_num 
by using the %type. Output l_num_vartype = 1.12
*/
create table deparments
( dept_id number not null primary key,
dept_name varchar2(60));


declare
	l_num number(5,2) not null default 2.21;
	l_num_vartype l_num%type := 1.123;  -- inherits data type and constant
	l_num_coltype departments.dept_id%type;
begin
	dbms_output.put_line(l_num);
	dbms_output.put_line(l_num_vartype);
	dbms_output.put_line(l_num_coltype);
end;

Error: Please load a database to perform operations

<h3>Timestamps</h3> <a class="anchor" id="section2.3"></a>

Timestamps have the ability to store fractional seconds with precision up to the billionth place (0.000000001).

In [10]:
select * from nls_instance_parameters
where parameter = 'nls_date_format';

Error: Please load a database to perform operations

In [11]:
l_date date := '10-nov-13';
l_date := to_date('10-nov-2013 15:25:34', 'dd-mon'rrrr hh24:mi:ss');
l_date := to_date('11/10/2013 15:25:34', 'mm/dd/rrrr hh24:mi:ss');

Error: Please load a database to perform operations

<h3>Interval Data Types</h3> <a class="anchor" id="section2.4"></a>

Below uses the interval to calculate the difference between two times while also preserving observation for daylight savings time. The `timestamp(2) with time zone` is being used to provide the time zone of that time, ‘PST’, following that is whether or not the zone is currently in daily sight savings time (‘DST’).

We see that if November. 3rd is when daily savings time ends, the difference between November 2 and November 4 results in 2 days and 1 hour.

In [12]:
declare
    l_tsmp_begin timestamp(2) with time zone :=
        to_timestamp_tz('02-nov-2013 10:00:00.0 pst pdt', 
                        'dd-mon-rrr hh24:mi:ss.ff tzr tzd');
    l_tsmp_end times timestamp(2) with time zone :=
        to_timestamp_tz('02-nov-2013 10:00:00.0 pst pdt', 
                            'dd-mon-rrr hh24:mi:ss.ff tzr tzd');
    
    int_ym interval year(3) to month;
    int_ds interval day(2) to second(2);
    
begin
    int_ds := l_tsmp_end - l_tsmp_begin;
    
    dbms_output.put_line('interval '||int_ds); --interval_day +01:00:00.00
end;

Error: Please load a database to perform operations

<h3>Booleans</h3> <a class="anchor" id="section2.5"></a>

Booleans in PL/SQL are not case-sensitive.

<ul>
    <li>True</li>
    <li>False</li>
    <li>Null</li>
</ul

<h3>Records</h3> <a class="anchor" id="section2.6"></a>

- Group of items
- Dot notation
- CANNOT be defined at the database level

See below usage: `TYPE rec_name IS RECORD`

In [16]:
create table departments
 (dept_id number not null primary key,
  dept_name varchar2(60));

declare
    type emp_rec is record ( emp_name varchar2(60),
                             dept_id deparments.dept_id%TYPE -- inheriting type of departments
                             loc varchar2(10) default 'CA');        
    l_emprec emp_rec;
begin
    l_emprec .emp_name := 'John';
    l_emprec .dept_id := 10;
    
    dbms_output.put_line('Employee Name is '||l_emprec .emp_name);
    dbms_output.put_line('Employee Location is '||l_emprec .loc);
end;

Error: Please load a database to perform operations

<h3>%ROWTYPE</h3> <a class="anchor" id="section2.7"></a>

- Table
- View
- Cursor

In [17]:
create table departments
    (dept_id number not null primary key,
     dept_name varchar2(60));

Error: Please load a database to perform operations

In [18]:
declare
    l_dept_rec departments%ROWTYPE;
begin
    l_dept_rec.dept_id := 10;
    dbms_output.put_line('Department ID is '||l_dept_rec.dept_id);
end;

Error: Please load a database to perform operations

<h2>Loops</h2> <a class="anchor" id="section3"></a>

<h3>For Loop</h3> <a class="anchor" id="section3.1"></a>

Below shows how for loops are implemented in PL/SQL. It also shows how steps can be used to count every 2, 3, … , n numbers. 

The `l_step := l_counter * 2` line is the step control. Multiplying each value in 1..3 loop by 2, gives a return of 2, 4, 6.

In [20]:
declare
    step_counter number;
begin
    for counter in 1..3 loop
        step_counter := counter * 2;
        dbms_output.put_line(step_counter)
    end loop;
end;

Error: Please load a database to perform operations

<h3>Loop Labels</h3> <a class="anchor" id="section3.2"></a>

In [21]:
begin
    <<outer>>
    for outer_counter in 1..3 loop
        dbms_output.put_line('outer_counter: '||outer_counter);
        <<inner>>
        for inner_counter in 1..5 loop
            exit outer when inner_counter = 3;
            dbms_output.put_line('inner counter: '||inner_counter);
        end loop inner;
    end loop outer;
end;

Error: Please load a database to perform operations

<h3>Nested Loops</h3> <a class="anchor" id="section3.3"></a>

Below shows the usage of a nested loop. There is an outer loop, and nested inside of that, an inner loop.

In [24]:
begin
    <<outer>>
    for outer_counter in 1..3 loop
        dbms_output.put_line('outer_counter: '||outer_counter);
        <<inner>>
        for inner_counter in 1..5 loop
            exit outer when inner_counter = 3;
            dbms_output.put_line('inner counter: '||inner_counter);
        end loop inner;
    end loop outer;
end;

Error: Please load a database to perform operations

<h4>Continue when</h4> <a class="anchor" id="section3.3.1"></a>

Below shows how an inner loop is terminated when a condition is satisfied. In this case, the CONTINUE outer WHEN is given a condition that l_inner_counter must equal 2 to break the inner loop and continue to the outer loop and continue the outer iteration. This effectively only prints the value of 1 inside of the inner loop.

Giving an output of: 1, 1, 2, 1, 3, 1

In [29]:
begin
    <<outer>>
    for outer_counter in 1..3 loop
        dbms_output.put_line('outer counter: '||outer_counter);
        <<inner>>
            for inner_counter in 1..3 loop
                continue outer when inner_counter = 2;
                dbms_output.put_line('inner_counter: '||inner_counter);
            end loop inner;
        dbms_output.put_line('outer counter at end: '||outer_counter);
    end loop outer;
end;

Error: Please load a database to perform operations

<h3>While Loops</h3> <a class="anchor" id="section3.4"></a>

<h4>Random Values</h4> <a class="anchor" id="section3.4.1"></a>

Below shows the usage of a while loop and `dmbs_random.value(1, 10)` to generate a random number between 1 and 10.

In [30]:
declare
    check int := 1;
begin
    while check < 5 loop
        check := dbms_random.value(1, 10);
        dbms_output.put_line(check);
    end loop;
end;

Error: Please load a database to perform operations

<h2>Conditional Exectuion</h2> <a class="anchor" id="section4"></a>

When a condition is met, a task is executed. 

**Note:** It is important to put the most probable conditions first in order to increase efficiency and to likely not demand exhaustion of all conditional statements.

<h3>If, Elsif, and Else Statements</h3><a class="anchor" id="section4.1"></a>
<p>If statements can be nested in PL/SQL like most other programming languages.</p>

If statements are formatted as `if <condition> then`

In [5]:
declare
    check int := 1;
begin
    if check = 1 then
        dbms_output.put_line(check);
    end if;
end;

Error: Please load a database to perform operations

Usage of `OR`

In [6]:
declare
    sales_amt number := 40000;
    no_of_orders number := 120;
    commission number;
begin
    if (sales_amt > 50000 and no_of_orders > 50)
        or -- OR statment allowing for multiple sufficient conditions
        (sales_amt < 50000 and no_of_ordrs > 100) then
        
        commission := 10
        dbms_output.put_line(commission);
    end if;
end;

Error: Please load a database to perform operations

Usage of `ELSE`

In [7]:
declare
    sales_amt number := 40000;
    no_of_orders number := 120;
    commission number;
begin
    if (sales_amt > 50000 and no_of_orders > 50)
        or -- OR statment allowing for multiple sufficient conditions
        (sales_amt < 50000 and no_of_ordrs > 100) then
        
        commission := 10
        dbms_output.put_line(commission);
    else
        commission := 5;
        dbms_output.put_line(commission);
    end if;
end;

Error: Please load a database to perform operations

Utilizing null evaluation. `NVL(variable, value)` evaluates a variable to see if it is null. If null, the variable is assigned the defined value. 

In [8]:
declare
    sales_amt number := 40000;
    no_of_orders number := 120;
    commission number;
begin
    if (nvl(sales_amt,0) > 50000 and no_of_orders > 50)
        or -- OR statment allowing for multiple sufficient conditions
        (nvl(sales_amt,0) < 50000 and no_of_ordrs > 100) then
        
        commission := 10
        dbms_output.put_line(commission);
    else
        commission := 5;
        dbms_output.put_line(commission);
    end if;
end;

Error: Please load a database to perform operations

Usage of `ELSIF`<br>
**Note:** else statements are not required to follow elsif.

In [9]:
declare
    sales_amt number := 40000;
    no_of_orders number := 120;
    commission number;
begin
    if (nvl(sales_amt,0) > 50000 and no_of_orders > 50) then
        commission := 10
        dbms_output.put_line(commission);
    elsif (nvl(sales_amt,0) < 50000 and no_of_orders > 100) then
        commission := 5;
        dbms_output.put_line(commission);
    else
        commission := 0;
        dbms_output.put_line(commission);
    end if;
end;

Error: Please load a database to perform operations

<h3>When Statements</h3><a class="anchor" id="section4.2"></a>

It is considered better practice to utilize Case/When statements instead of If/Then as it improves readability.

In [12]:
declare
    ticket_priority varchar2(8) := 'medium';
    support_tier number;
begin
    case ticket_priority
        when 'high' then
            support_tier := 1;
        when 'medium' then
            support_tier := 2;
        when 'low' then
            support_tier := 3;
        else
            support_tier := 0;
    end case;
    dbms_output.put_line(ticket_priority||': '||support_tier);
end;

Error: Please load a database to perform operations

<h3>Simple Case Expression</h3><a class="anchor" id="section4.3"></a>

Below shows how to assign a value to a variable based on condition statements.<br><br>
**Note:** the return value assigned to the variable must match the assignment type under the declare statement. So below, support_tier number, must be assigned a number value.

In [15]:
declare
    ticket_priority varchar2(8) := 'medium';
    support_tier number;
begin
    support_tier :=
        case ticket_priority
            when 'high' then 1
            when 'medium' then 2
            when 'low' then 3
            else 0
        end;
    dbms_output.put_line(support_tier)
end;

Error: Please load a database to perform operations

<h2>Cursors</h2><a class="anchor" id="section5"></a>

Lifecycle of a cursor:
- Open state:
    1. Area memory is assigned
    2. SQL statement parsed and binded
    3. Statement executed
    4. Pointer moved to the first row
- Fetch state:
    5. row fetched
- Close phase:
    6. memory released

In [20]:
create table departments
(dept_id number not null primary key,
 dept_name varchar2(60));

Error: Please load a database to perform operations

Below creates a constraint using <variable_name>_fk to annotate it as a foreign key

In [21]:
create table employee
(emp_id number not null primary key,
 emp_name varchar2(60),
 emp_dept_id number,
 emp_loc varchar2(60),
 emp_sal number,
 constraint emp_dept_fk foreign key(emp_dept_id)
 references departments(dept_id));

Error: Please load a database to perform operations

<h3>Implicit Cursors</h3><a class="anchor" id="section5.1"></a>

Implicit cursors are automatically created. <br>

Attributes:
- `sql%found` --> true if a row is affected
- `sql%notfound` --> true if no rows are affected
- `sql%rowcount` --> number of rows affected

In [25]:
declare
    l_dept_id departments.dept_id%type;
    l_dept_name departments.dept_name%type;
begin
    select dept_id, dept_name
        into l_dept_id, l_dept_name
    from departments
    where dept_id = 1;
    if sql%found then
        dbms_output.put_line(sql%rowcount);
    end if;
end;

Error: Please load a database to perform operations

<h3>Explicit Cursors</h3><a class="anchor" id="section5.2"></a>

Explicit cursors are not automatically created. <br>

- should be declared in `declare` section
- more control over the context area (memory area)